###  py

In [4]:
# Corrected Imports and Initial Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from plot_functions import plot_graph, plot_histogram  # Assuming plot functions are defined
from io import BytesIO
from PIL import Image

# Step 1: Create a Fake DataFrame for example
data = {
    'Date': pd.date_range(start='2024-01-01', periods=100, freq='D'),
    'Stock A': np.random.normal(100, 10, 100),
    'Stock B': np.random.normal(200, 15, 100),
    'Stock C': np.random.normal(150, 20, 100),
    'Volume A': np.random.randint(1000, 5000, 100),
    'Volume B': np.random.randint(2000, 7000, 100),
    'Volume C': np.random.randint(1500, 6000, 100),
}
df = pd.DataFrame(data)

# Initialize global variables
slides = []
selected_slide_index = 0
plot_registry = {
    "plot_graph": {
        "function": plot_graph,
        "params": {"color": "blue"}
    },
    "plot_histogram": {
        "function": plot_histogram,
        "params": {"color": "blue"}
    },
}

# Sidebar, main display, and customization window layout
left_sidebar = widgets.VBox()
main_slide_display = widgets.Output(layout=widgets.Layout(width="60%", border="1px solid black"))
customization_window = widgets.VBox(layout=widgets.Layout(width="20%", border="1px solid black"))

# Generate Slide Thumbnail Function
def generate_slide_thumbnail(slide):
    """Generate a thumbnail of the slide's plots as an image."""
    fig, axs = plt.subplots(2, 2, figsize=(3, 3))  # Create a small 2x2 grid for the thumbnail

    # Plot each item on its position within the 2x2 grid
    for plot_data in slide["plots"]:
        ax = axs[0][0] if plot_data["position"] == "top-left" else \
             axs[0][1] if plot_data["position"] == "top-right" else \
             axs[1][0] if plot_data["position"] == "bottom-left" else \
             axs[1][1]  # bottom-right default
        plot_function = plot_registry[plot_data["function_name"]]["function"]
        plot_function(ax, **plot_data["params"])
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()

    # Convert the plot to an image and resize it to thumbnail size
    buf = BytesIO()
    fig.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    img.thumbnail((100, 100))  # Resize to a small thumbnail
    return img

# Plotting and Customization Functions
def update_custom_plot():
    """Update the plot in the customization window based on current widget values."""
    # Get the function to plot from the registry
    plot_function = plot_registry[plot_type_dropdown.value]["function"]

    # Collect the current parameter values
    x_data = df[x_column_dropdown.value]
    y_data = df[y_column_dropdown.value]
    color = color_picker.value

    # Clear the output widget and display the updated plot
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        if plot_type_dropdown.value == "plot_histogram":
            ax.hist(y_data, bins=30, color=color, alpha=0.7)
            ax.set_xlabel(y_column_dropdown.value)
            ax.set_title(f"Histogram of {y_column_dropdown.value}")
        else:
            ax.plot(x_data, y_data, color=color)
            ax.set_xlabel(x_column_dropdown.value)
            ax.set_ylabel(y_column_dropdown.value)
            ax.set_title(f"{y_column_dropdown.value} vs {x_column_dropdown.value}")
        plt.show()

def open_customization_window():
    """Displays the customization window with controls and a live-updating plot."""
    global color_picker, plot_output, position_dropdown, plot_type_dropdown, x_column_dropdown, y_column_dropdown

    # Dropdown for plot position selection
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"],
                                         description="Position")

    # Dropdown for plot type selection
    plot_type_dropdown = widgets.Dropdown(options=list(plot_registry.keys()), description="Plot Type:")

    # Dropdown for selecting X and Y columns
    x_column_dropdown = widgets.Dropdown(options=df.columns, description='X Axis:')
    y_column_dropdown = widgets.Dropdown(options=df.columns, description='Y Axis:')

    # Color Picker
    color_picker = widgets.ColorPicker(value="blue", description="Color")

    # Plot Output
    plot_output = widgets.Output()

    # Confirm button to add plot to slide
    confirm_button = widgets.Button(description="Confirm Plot Addition")
    confirm_button.on_click(lambda b: add_plot_to_slide(position_dropdown.value, plot_type_dropdown.value))

    # Attach observers to dynamically update the plot in the customization window
    plot_type_dropdown.observe(lambda change: update_custom_plot(), names="value")
    x_column_dropdown.observe(lambda change: update_custom_plot(), names="value")
    y_column_dropdown.observe(lambda change: update_custom_plot(), names="value")
    color_picker.observe(lambda change: update_custom_plot(), names="value")

    # Initial plot rendering
    update_custom_plot()

    # Display the customization window with the controls
    customization_window.children = [
        position_dropdown,
        plot_type_dropdown,
        x_column_dropdown,
        y_column_dropdown,
        color_picker,
        plot_output,
        confirm_button
    ]

# Plot Management: Adding and Rendering
def add_plot_to_slide(position, plot_type):
    """Add the plot currently visible in the customization window to the slide."""
    x_data = df[x_column_dropdown.value]
    y_data = df[y_column_dropdown.value]
    color = color_picker.value

    new_plot = {
        "function_name": plot_type,
        "params": {
            "x_data": x_data,
            "y_data": y_data,
            "color": color
        },
        "position": position
    }

    slides[selected_slide_index]["plots"].append(new_plot)
    render_slide()
    update_sidebar()

def render_slide():
    """Render the plots in the selected slide."""
    main_slide_display.clear_output(wait=True)
    with main_slide_display:
        slide = slides[selected_slide_index]
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        position_map = {
            "top-left": (0, 0), "top-right": (0, 1),
            "bottom-left": (1, 0), "bottom-right": (1, 1)
        }
        for plot_data in slide["plots"]:
            pos = position_map[plot_data["position"]]
            ax = axs[pos]
            if plot_data["function_name"] == "plot_histogram":
                ax.hist(plot_data["params"]["y_data"], bins=30, color=plot_data["params"]["color"], alpha=0.7)
                ax.set_title(f"Histogram of {y_column_dropdown.value}")
            else:
                ax.plot(plot_data["params"]["x_data"], plot_data["params"]["y_data"], color=plot_data["params"]["color"])
                ax.set_xlabel(x_column_dropdown.value)
                ax.set_ylabel(y_column_dropdown.value)
                ax.set_title(f"{y_column_dropdown.value} vs {x_column_dropdown.value}")
        plt.tight_layout()
        plt.show()

# Select Slide
def select_slide(index):
    global selected_slide_index
    selected_slide_index = index
    render_slide()

# Sidebar Update
def update_sidebar():
    """Update the sidebar to show slide icons or labels for each slide."""
    sidebar_content = []
    for i, slide in enumerate(slides):
        # Generate the thumbnail for the slide
        thumbnail_img = generate_slide_thumbnail(slide)
        
        # Convert the thumbnail image to a widget
        with BytesIO() as output:
            thumbnail_img.save(output, format="PNG")
            img_widget = widgets.Image(value=output.getvalue(), format='png', width=100, height=100)
        
        # Label the slide and combine with the image
        slide_label = widgets.Label(f"Slide {i + 1}")
        
        # Combine the thumbnail and label into a VBox
        slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
            width="120px", height="150px", border="1px solid black", align_items="center"
        ))
        
        # Set the click event to select the slide
        slide_button_box = widgets.Button(description=f"Select Slide {i + 1}")
        slide_button_box.on_click(lambda b, idx=i: select_slide(idx))
        
        # Add button and thumbnail VBox
        sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

    left_sidebar.children = sidebar_content

# Application Layout
def display_app_layout():
    update_sidebar()
    render_slide()
    open_customization_window()
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))

# Initialize the application with a single slide
slides.append({"uid": "slide_1", "plots": []})
display_app_layout()

# Control buttons
add_slide_button = widgets.Button(description="Add Slide")
add_slide_button.on_click(lambda b: slides.append({"uid": f"slide_{len(slides) + 1}", "plots": []}) or update_sidebar())

export_button = widgets.Button(description="Export to PPT")
# Add export functionality if needed

display(widgets.HBox([add_slide_button, export_button]))


In [7]:
# Corrected Imports and Initial Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from plot_functions import plot_graph, plot_histogram  # Assuming plot functions are defined
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator

# Assuming create_multi_index_dataframe is already defined
# Create the sample DataFrame
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]  # Maturity in months
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)

# Initialize global variables for slide application
slides = []
selected_slide_index = 0
plot_registry = {
    "plot_graph": {
        "function": plot_graph,
        "params": {"color": "blue"}
    },
    "plot_histogram": {
        "function": plot_histogram,
        "params": {"color": "blue"}
    },
}

# Sidebar, main display, and customization window layout
left_sidebar = widgets.VBox()
main_slide_display = widgets.Output(layout=widgets.Layout(width="60%", border="1px solid black"))
customization_window = widgets.VBox(layout=widgets.Layout(width="20%", border="1px solid black"))

# Create Widgets for the Customization Page
udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
window_widget = widgets.Dropdown(options=[52*5, 52*5*2, 52*5*3], value=52*5, description='Window:')
plot_type_dropdown = widgets.Dropdown(options=list(plot_registry.keys()), description="Plot Type:")
color_picker = widgets.ColorPicker(value="blue", description="Color")
plot_output = widgets.Output()

# Generate Slide Thumbnail Function
def generate_slide_thumbnail(slide):
    """Generate a thumbnail of the slide's plots as an image."""
    fig, axs = plt.subplots(2, 2, figsize=(3, 3))  # Create a small 2x2 grid for the thumbnail

    # Plot each item on its position within the 2x2 grid
    for plot_data in slide["plots"]:
        ax = axs[0][0] if plot_data["position"] == "top-left" else \
             axs[0][1] if plot_data["position"] == "top-right" else \
             axs[1][0] if plot_data["position"] == "bottom-left" else \
             axs[1][1]  # bottom-right default
        plot_function = plot_registry[plot_data["function_name"]]["function"]
        plot_function(ax, **plot_data["params"])
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()

    # Convert the plot to an image and resize it to thumbnail size
    buf = BytesIO()
    fig.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    img.thumbnail((100, 100))  # Resize to a small thumbnail
    return img

# Function to filter the DataFrame based on widget selections
def filter_dataframe():
    """Filter the DataFrame based on user selections."""
    selected_udl = list(udl_widget.value)
    selected_matu = list(matu_widget.value)
    selected_param = list(param_widget.value)
    selected_level = list(level_widget.value)

    try:
        filtered_df = df.loc[:, df.columns.get_level_values('UDL').isin(selected_udl) &
                             df.columns.get_level_values('Matu').isin(selected_matu) &
                             df.columns.get_level_values('Param').isin(selected_param) &
                             df.columns.get_level_values('Level').isin(selected_level)]
        return filtered_df
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

# Update the Custom Plot Function
def update_custom_plot():
    """Update the plot in the customization window based on current widget values."""
    # Get the function to plot from the registry
    plot_function = plot_registry[plot_type_dropdown.value]["function"]

    # Collect the current parameter values
    filtered_df = filter_dataframe()  # Use filtered DataFrame based on selected criteria
    if filtered_df.empty:
        with plot_output:
            clear_output()
            print("No valid data available for the selected combination. Please adjust your selection.")
        return

    x_data = filtered_df.index
    y_data = filtered_df.mean(axis=1)  # Averaging across the selected columns to get a single plot

    color = color_picker.value

    # Clear the output widget and display the updated plot
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        if plot_type_dropdown.value == "plot_histogram":
            ax.hist(y_data, bins=30, color=color, alpha=0.7)
            ax.set_xlabel("Value")
            ax.set_title(f"Histogram Plot")
        else:
            ax.plot(x_data, y_data, color=color)
            ax.set_xlabel("Date")
            ax.set_ylabel("Value")
            ax.set_title(f"Time Series Plot")
        plt.show()

# Open Customization Window
def open_customization_window():
    """Displays the customization window with controls and a live-updating plot."""
    # Attach observers to dynamically update the plot in the customization window
    udl_widget.observe(lambda change: update_custom_plot(), names="value")
    matu_widget.observe(lambda change: update_custom_plot(), names="value")
    param_widget.observe(lambda change: update_custom_plot(), names="value")
    level_widget.observe(lambda change: update_custom_plot(), names="value")
    window_widget.observe(lambda change: update_custom_plot(), names="value")
    plot_type_dropdown.observe(lambda change: update_custom_plot(), names="value")
    color_picker.observe(lambda change: update_custom_plot(), names="value")

    # Initial plot rendering
    update_custom_plot()

    # Display the customization window with the controls
    customization_window.children = [
        udl_widget, matu_widget, param_widget, level_widget, window_widget,
        plot_type_dropdown, color_picker, plot_output
    ]

# Plot Management: Adding and Rendering
def add_plot_to_slide(position, plot_type):
    """Add the plot currently visible in the customization window to the slide."""
    x_data = filter_dataframe().index
    y_data = filter_dataframe().mean(axis=1)
    color = color_picker.value

    if plot_type == "plot_histogram":
        new_plot = {
            "function_name": plot_type,
            "params": {
                "y_data": y_data,
                "color": color
            },
            "position": position
        }
    else:
        new_plot = {
            "function_name": plot_type,
            "params": {
                "x_data": x_data,
                "y_data": y_data,
                "color": color
            },
            "position": position
        }

    slides[selected_slide_index]["plots"].append(new_plot)
    render_slide()
    update_sidebar()

# The rest of the functions and app layout


# Initialize the application with a single slide
slides.append({"uid": "slide_1", "plots": []})
display_app_layout()

# Control buttons
add_slide_button = widgets.Button(description="Add Slide")
add_slide_button.on_click(lambda b: slides.append({"uid": f"slide_{len(slides) + 1}", "plots": []}) or update_sidebar())

export_button = widgets.Button(description="Export to PPT")
# Note: Add export functionality if needed

display(widgets.HBox([add_slide_button, export_button]))


In [9]:
# Corrected Imports and Initial Setup
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    """
    Creates a DataFrame with a multi-level column index based on the given inputs.
    """
    arrays = [(u, m, p, l) for u in udl for m in matu for p in param for l in level[p]]
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    return pd.DataFrame(data=data, index=time_index, columns=multi_index)

# Step 2: Create functions for rolling metrics
def rolling_percentile_rank(df, window=52, time_series=True):
    if time_series:
        return df.apply(lambda x: x.rolling(window=window).apply(lambda y: y.rank(pct=True).iloc[-1] if len(y.dropna()) > 0 else np.nan, raw=False))
    else:
        return df.iloc[-window:].rank(pct=True)

def rolling_z_score(df, window=52, time_series=True):
    if time_series:
        rolling_mean = df.rolling(window=window).mean()
        rolling_std = df.rolling(window=window).std()
        return (df - rolling_mean) / rolling_std
    else:
        mean = df.iloc[-window:].mean()
        std = df.iloc[-window:].std()
        return (df.iloc[-1] - mean) / std

# Step 3: Create the main application
def create_app(df):
    """
    Creates an interactive application to compute rolling metrics and visualize the results.
    """

    # Widgets for user input
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52*5, 52*5*2, 52*5*3], value=52*5, description='Window:')
    metric_widget = widgets.Dropdown(options=['z_score', 'percentile_rank'], value='z_score', description='Metric:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    color_picker = widgets.ColorPicker(value="blue", description="Color")
    plot_type_dropdown = widgets.Dropdown(options=['plot_graph', 'plot_histogram'], description="Plot Type:")
    plot_button = widgets.Button(description='Plot Result')
    output_plot = widgets.Output()
    output_table = widgets.Output()

    # Function to filter the dataframe based on widget values
    def filter_dataframe():
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        return df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                      df.columns.get_level_values('Matu').isin(selected_matu) &
                      df.columns.get_level_values('Param').isin(selected_param) &
                      df.columns.get_level_values('Level').isin(selected_level)]

    # Function to update the plot in the customization window
    def update_custom_plot():
        with output_plot:
            clear_output(wait=True)
            filtered_df = filter_dataframe()  # Use filtered DataFrame based on selected criteria
            if filtered_df.empty:
                print("No data available for the selected combination.")
                return
            fig, ax = plt.subplots(figsize=(5, 4))
            if plot_type_dropdown.value == "plot_histogram":
                ax.hist(filtered_df.values.flatten(), bins=30, color=color_picker.value, alpha=0.7)
                ax.set_title(f"Histogram of Selected Data")
            else:
                filtered_df.mean(axis=1).plot(ax=ax, color=color_picker.value)
                ax.set_title(f"Line Plot of Selected Data")
            plt.show()

    # Function to add the plot to the slide
    def add_plot_to_slide(position, plot_type):
        filtered_df = filter_dataframe()
        if filtered_df.empty:
            print("No valid data available to add to slide.")
            return

        plot_params = {"color": color_picker.value}
        if plot_type == "plot_histogram":
            plot_params["y_data"] = filtered_df.values.flatten()
        else:
            plot_params["x_data"] = filtered_df.index
            plot_params["y_data"] = filtered_df.mean(axis=1)

        new_plot = {
            "function_name": plot_type,
            "params": plot_params,
            "position": position
        }
        slides[selected_slide_index]["plots"].append(new_plot)
        render_slide()
        update_sidebar()

    # Attach the click event to the plot button
    plot_button.on_click(lambda b: update_custom_plot())

    # Attach event to add plot to slide
    confirm_button = widgets.Button(description="Add Plot to Slide")
    confirm_button.on_click(lambda b: add_plot_to_slide(position_dropdown.value, plot_type_dropdown.value))

    # Dropdown for plot position selection
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position")

    # Customization Window Layout
    customization_window.children = [
        udl_widget, matu_widget, param_widget, level_widget, window_widget,
        metric_widget, start_date_widget, end_date_widget,
        plot_type_dropdown, color_picker, plot_button, output_plot, confirm_button
    ]

    # Display the application layout
    display_app_layout()

# Application Layout
def display_app_layout():
    update_sidebar()
    render_slide()
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))

# Create Multi-Index DataFrame
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25],
    'Moneyness': [10, 20, 30, 40, 50]
}
df = create_multi_index_dataframe(udl, matu, param, level)

# Initialize Slides
slides = [{"uid": "slide_1", "plots": []}]
selected_slide_index = 0
left_sidebar = widgets.VBox()
main_slide_display = widgets.Output(layout=widgets.Layout(width="60%", border="1px solid black"))
customization_window = widgets.VBox(layout=widgets.Layout(width="20%", border="1px solid black"))



In [14]:
# Updated Customization Window Code

# Updated Imports and Setup from previous example
from plot_functions import plot_graph, plot_histogram  # Assuming plot functions are defined
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
from PIL import Image

# Assume df and slides are already defined from your original code
df = pd.DataFrame({
    'Date': pd.date_range(start='2024-01-01', periods=100, freq='D'),
    'Stock A': np.random.normal(100, 10, 100),
    'Stock B': np.random.normal(200, 15, 100),
    'Stock C': np.random.normal(150, 20, 100),
    'Volume A': np.random.randint(1000, 5000, 100),
    'Volume B': np.random.randint(2000, 7000, 100),
    'Volume C': np.random.randint(1500, 6000, 100),
})

slides = [{"uid": "slide_1", "plots": []}]
selected_slide_index = 0

# Customization Window
def open_customization_window():
    """Displays the customization window with controls and a live-updating plot."""
    global color_picker, plot_output, position_dropdown, plot_type_dropdown, x_column_dropdown, y_column_dropdown, start_date_picker, end_date_picker

    # Dropdown for plot position selection
    position_dropdown = widgets.Dropdown(
        options=["top-left", "top-right", "bottom-left", "bottom-right"],
        description="Position"
    )

    # Dropdown for plot type selection
    plot_type_dropdown = widgets.Dropdown(
        options=list(plot_registry.keys()),
        description="Plot Type:"
    )

    # Dropdown for selecting X and Y columns
    x_column_dropdown = widgets.Dropdown(
        options=df.columns, 
        description='X Axis:'
    )
    y_column_dropdown = widgets.Dropdown(
        options=df.columns, 
        description='Y Axis:'
    )

    # Date Picker Widgets for start and end date
    start_date_picker = widgets.DatePicker(
        description='Start Date:', 
        value=df['Date'].min()
    )
    end_date_picker = widgets.DatePicker(
        description='End Date:', 
        value=df['Date'].max()
    )

    # Color Picker
    color_picker = widgets.ColorPicker(
        value="blue", 
        description="Color"
    )

    # Plot Output
    plot_output = widgets.Output()

    # Confirm button to add plot to slide
    confirm_button = widgets.Button(description="Confirm Plot Addition")
    confirm_button.on_click(lambda b: add_plot_to_slide(
        position_dropdown.value, 
        plot_type_dropdown.value,
        start_date_picker.value,
        end_date_picker.value
    ))

    # Attach observers to dynamically update the plot in the customization window
    plot_type_dropdown.observe(lambda change: update_custom_plot(), names="value")
    x_column_dropdown.observe(lambda change: update_custom_plot(), names="value")
    y_column_dropdown.observe(lambda change: update_custom_plot(), names="value")
    color_picker.observe(lambda change: update_custom_plot(), names="value")
    start_date_picker.observe(lambda change: update_custom_plot(), names="value")
    end_date_picker.observe(lambda change: update_custom_plot(), names="value")

    # Initial plot rendering
    update_custom_plot()

    # Display the customization window with the controls
    customization_window.children = [
        position_dropdown,
        plot_type_dropdown,
        x_column_dropdown,
        y_column_dropdown,
        start_date_picker,
        end_date_picker,
        color_picker,
        plot_output,
        confirm_button
    ]

# Update the add_plot_to_slide function to handle start and end date range
def add_plot_to_slide(position, plot_type, start_date, end_date):
    """Add the plot currently visible in the customization window to the slide."""
    x_data = df[(df['Date'] >= pd.Timestamp(start_date)) & (df['Date'] <= pd.Timestamp(end_date))][x_column_dropdown.value]
    y_data = df[(df['Date'] >= pd.Timestamp(start_date)) & (df['Date'] <= pd.Timestamp(end_date))][y_column_dropdown.value]
    color = color_picker.value

    new_plot = {
        "function_name": plot_type,
        "params": {
            "x_data": x_data,
            "y_data": y_data,
            "color": color
        },
        "position": position
    }

    slides[selected_slide_index]["plots"].append(new_plot)
    render_slide()
    update_sidebar()

# Update the update_custom_plot function to handle new date picker inputs
def update_custom_plot():
    """Update the plot in the customization window based on current widget values."""
    # Get the function to plot from the registry
    plot_function = plot_registry[plot_type_dropdown.value]["function"]

    # Collect the current parameter values
    x_data = df[(df['Date'] >= pd.Timestamp(start_date_picker.value)) & (df['Date'] <= pd.Timestamp(end_date_picker.value))][x_column_dropdown.value]
    y_data = df[(df['Date'] >= pd.Timestamp(start_date_picker.value)) & (df['Date'] <= pd.Timestamp(end_date_picker.value))][y_column_dropdown.value]
    color = color_picker.value

    # Clear the output widget and display the updated plot
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        if plot_type_dropdown.value == "plot_histogram":
            ax.hist(y_data, bins=30, color=color, alpha=0.7)
            ax.set_xlabel(y_column_dropdown.value)
            ax.set_title(f"Histogram of {y_column_dropdown.value}")
        else:
            ax.plot(x_data, y_data, color=color)
            ax.set_xlabel(x_column_dropdown.value)
            ax.set_ylabel(y_column_dropdown.value)
            ax.set_title(f"{y_column_dropdown.value} vs {x_column_dropdown.value}")
        plt.show()

# Initialize the application and layout
slides = [{"uid": "slide_1", "plots": []}]
open_customization_window()


In [15]:

# Run the app
create_app(df)

KeyError: 'Requested level (UDL) does not match index name (None)'

In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    """
    Creates a DataFrame with a multi-level column index based on the given inputs.

    Parameters:
    udl (list): List of underlying assets.
    matu (list): List of maturities (including 'None' for spot).
    param (list): List of parameters (e.g., 'Spot', 'Delta', 'Moneyness').
    level (dict): Dictionary specifying levels for each parameter.
    start_date (str): Start date for the time index.
    periods (int): Number of periods for the time index.
    freq (str): Frequency for the time index.

    Returns:
    pd.DataFrame: DataFrame with multi-index columns and time index.
    """
    # Create MultiIndex columns
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))

    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])

    # Create a time index for the DataFrame
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)

    # Create the DataFrame with consistent data to reduce NaNs
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)

    return df

# Step 2: Create functions for rolling metrics
def rolling_percentile_rank(df, window=52, time_series=True):
    """
    Computes the rolling percentile rank over a specified window.

    Parameters:
    df (pd.DataFrame): The input DataFrame with a multi-level column index.
    window (int): Rolling window size in business days.
    time_series (bool): If True, computes percentile rank along the time index for each column;
                        If False, computes percentile rank along the last available date across columns.

    Returns:
    pd.DataFrame: DataFrame with rolling percentile rank.
    """
    if time_series:
        return df.apply(lambda x: x.rolling(window=window).apply(lambda y: y.rank(pct=True).iloc[-1] if len(y.dropna()) > 0 else np.nan, raw=False))
    else:
        return df.iloc[-window:].rank(pct=True)

def rolling_z_score(df, window=52, time_series=True):
    """
    Computes the rolling z-score over a specified window.

    Parameters:
    df (pd.DataFrame): The input DataFrame with a multi-level column index.
    window (int): Rolling window size in business days.
    time_series (bool): If True, computes z-score along the time index for each column;
                        If False, computes z-score along the last available date across columns.

    Returns:
    pd.DataFrame: DataFrame with rolling z-score.
    """
    if time_series:
        rolling_mean = df.rolling(window=window).mean()
        rolling_std = df.rolling(window=window).std()
        return (df - rolling_mean) / rolling_std
    else:
        mean = df.iloc[-window:].mean()
        std = df.iloc[-window:].std()
        return (df.iloc[-1] - mean) / std

# Step 3: Create the main application
def create_app(df):
    """
    Creates an interactive application to compute rolling metrics and visualize the results.

    Parameters:
    df (pd.DataFrame): The input DataFrame with a multi-level column index.
    """
    # Widgets for user input
    udl_default = df.columns.get_level_values('UDL').unique()[0]
    matu_default = df.columns.get_level_values('Matu').unique()[0]
    param_default = df.columns.get_level_values('Param').unique()[0]
    level_default = df.columns.get_level_values('Level').unique()[0]

    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[udl_default], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[matu_default], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[param_default], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[level_default], description='Level:')
    window_widget = widgets.Dropdown(options=[52*5, 52*5*2, 52*5*3], value=52*5, description='Window:')
    metric_widget = widgets.Dropdown(options=['z_score', 'percentile_rank'], value='z_score', description='Metric:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Plot Result')
    output_plot = widgets.Output()
    output_table = widgets.Output()

    # Function to update and plot
    def on_plot_button_clicked(b):
        with output_plot:
            clear_output()  # Clear previous output to avoid clutter
            selected_udl = list(udl_widget.value)
            selected_matu = list(matu_widget.value)
            selected_param = list(param_widget.value)
            selected_level = list(level_widget.value)
            selected_window = window_widget.value
            metric = metric_widget.value
            start_date = start_date_widget.value
            end_date = end_date_widget.value

            # Ensure that at least one option is selected for each category
            if not selected_udl or not selected_matu or not selected_param or not selected_level:
                print("Please select at least one option from UDL, Matu, Param, and Level.")
                return

            # Debug prints to understand the filtering
            print(f"Selected UDL: {selected_udl}")
            print(f"Selected Matu: {selected_matu}")
            print(f"Selected Param: {selected_param}")
            print(f"Selected Level: {selected_level}")
            print(f"Selected Window: {selected_window}")
            print(f"Selected Metric: {metric}")
            print(f"Selected Start Date: {start_date}")
            print(f"Selected End Date: {end_date}")

            # Filter DataFrame based on selections
            try:
                filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                                   df.columns.get_level_values('Matu').isin(selected_matu) &
                                                   df.columns.get_level_values('Param').isin(selected_param) &
                                                   df.columns.get_level_values('Level').isin(selected_level)]
                # Debug prints after filtering
                print(f"Filtered DataFrame Columns: {filtered_df.columns}")
                if filtered_df.isna().all().all() or filtered_df.empty:
                    print("No valid data available for the selected combination. Please adjust your selection.")
                    return

                # Apply computation based on selected metric
                if metric == 'z_score':
                    result_df = rolling_z_score(filtered_df, window=selected_window, time_series=True)
                elif metric == 'percentile_rank':
                    result_df = rolling_percentile_rank(filtered_df, window=selected_window, time_series=True)

                # Check if the resulting DataFrame is empty
                if result_df.empty or result_df.isna().all().all():
                    print("Resulting DataFrame is empty after applying the rolling operation. Please adjust your parameters or window size.")
                    return

                # Replace NaN values with zeros for visualization purposes
                result_df = result_df.fillna(0)

                # Plotting with Plotly
                reset_df = result_df.reset_index()
                fig = go.Figure()

                for column in result_df.columns:
                    fig.add_trace(go.Scatter(
                        x=reset_df['index'],
                        y=reset_df[column],
                        mode='lines',
                        name=str(column)
                    ))

                fig.update_layout(title=f'{metric.capitalize()} Plot for Window {selected_window} Days',
                                  xaxis_title='Date', yaxis_title='Value')
                fig.show()

                # Display the table output
                with output_table:
                    clear_output()  # Clear previous table output
                    display(filtered_df)

            except Exception as e:
                print(f"An error occurred: {e}")

    # Attach the click event to the button
    plot_button.on_click(on_plot_button_clicked)

    # Display the widgets and output
    display(widgets.VBox([udl_widget, matu_widget, param_widget, level_widget, window_widget, metric_widget, start_date_widget, end_date_widget, plot_button, widgets.HBox([output_plot, output_table])]))

# Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]  # Maturity in months
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)

# Create the app
create_app(df)


In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    """
    Creates a DataFrame with a multi-level column index based on the given inputs.
    """
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Rolling functions for calculations
def rolling_z_score(df, window=52):
    rolling_mean = df.rolling(window=window).mean()
    rolling_std = df.rolling(window=window).std()
    return (df - rolling_mean) / rolling_std

# Step 3: The main app with customizable plotting and placement
def create_app(df):
    # Widgets for user input
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot')
    add_to_slide_button = widgets.Button(description='Add to Slide', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0

    # Function to add plot to a slide
    def add_plot_to_slide(position, plot_data):
        slides[selected_slide_index]["plots"].append({"position": position, "data": plot_data})
        update_sidebar()
        render_slide()

    # Sidebar update
    left_sidebar = widgets.VBox()
    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            thumbnail = generate_slide_thumbnail(slide)
            img_widget = widgets.Image(value=thumbnail, format='png', width=100, height=100)
            slide_label = widgets.Label(f"Slide {i + 1}")
            slide_button = widgets.VBox([img_widget, slide_label])
            sidebar_content.append(slide_button)
        left_sidebar.children = sidebar_content

    # Function to generate slide thumbnail
    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        for plot_data in slide["plots"]:
            ax = axs[0][0] if plot_data["position"] == "top-left" else \
                 axs[0][1] if plot_data["position"] == "top-right" else \
                 axs[1][0] if plot_data["position"] == "bottom-left" else \
                 axs[1][1]
            ax.plot(plot_data["data"].index, plot_data["data"].values)
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()
        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        return buf.read()

    # Function to render the current slide
    main_slide_display = widgets.Output(layout=widgets.Layout(width="60%", border="1px solid black"))
    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for plot_data in slide["plots"]:
                pos = (0, 0) if plot_data["position"] == "top-left" else \
                      (0, 1) if plot_data["position"] == "top-right" else \
                      (1, 0) if plot_data["position"] == "bottom-left" else \
                      (1, 1)
                ax = axs[pos]
                ax.plot(plot_data["data"].index, plot_data["data"].values)
            plt.tight_layout()
            plt.show()

    # Function to preview the plot
    def on_plot_button_clicked(b):
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        # Filter DataFrame
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                return

        # Calculate rolling z-score
        result_df = rolling_z_score(filtered_df, window=selected_window).fillna(0)

        # Plot Preview
        with output_plot:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(6, 4))
            for column in result_df.columns:
                ax.plot(result_df.index, result_df[column], label=str(column))
            ax.set_title("Preview of Selected Plot")
            ax.legend()
            plt.show()

        # Enable "Add to Slide" button
        add_to_slide_button.disabled = False

    # Function to add previewed plot to slide
    def on_add_to_slide_button_clicked(b):
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value
        position = position_dropdown.value

        # Filter DataFrame again
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        result_df = rolling_z_score(filtered_df, window=selected_window).fillna(0)
        add_plot_to_slide(position, result_df)

    # Attach button click events
    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    # Display the UI
    customization_window = widgets.VBox([udl_widget, matu_widget, param_widget, level_widget,
                                         window_widget, start_date_widget, end_date_widget,
                                         position_dropdown, plot_button, add_to_slide_button, output_plot])

    # Full app layout
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})
    render_slide()

# Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [1]:
import plots
from plots import *

### Save

In [43]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    """
    Creates a DataFrame with a multi-level column index based on the given inputs.
    """
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Rolling functions for calculations
def rolling_z_score(df, window=52):
    rolling_mean = df.rolling(window=window).mean()
    rolling_std = df.rolling(window=window).std()
    return (df - rolling_mean) / rolling_std

# Step 3: Create the custom stats chart function
def create_stats_chart(df):
    """
    Create a custom stats chart based on the provided data.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Dummy data for demonstration; replace with your actual data calculation logic
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plot 20th-80th percentile range as bars
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

    # Plot min/max as horizontal lines
    for i, category in enumerate(categories):
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

    # Plot average as triangles
    for i, category in enumerate(categories):
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

    # Plot last values as diamonds
    for i, category in enumerate(categories):
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

    # Customization
    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.set_title("Custom Stats Chart")
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Custom legend
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, frameon=False)

    plt.tight_layout()
    plt.show()

# Step 4: The main app with customizable plotting and placement
def create_app(df):
    # Widgets for user input
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=[df.columns.get_level_values('UDL').unique()[0]], description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0

    # Sidebar and Main Slide Display
    left_sidebar = widgets.VBox()
    main_slide_display = widgets.Output(layout=widgets.Layout(width="55%", height="600px", border="1px solid black"))

    # Function to preview the plot
    def on_plot_button_clicked(b):
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        # Filter DataFrame
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                return

        # Plot Preview based on selected plot type
        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = rolling_z_score(filtered_df, window=selected_window).fillna(0)
                fig, ax = plt.subplots(figsize=(6, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
                plt.show()
            elif plot_type == "Custom Stats Chart":
                create_stats_chart(filtered_df)

        # Enable "Add to Slide" button
        add_to_slide_button.disabled = False

    # Attach button click events
    plot_button.on_click(on_plot_button_clicked)

    # Display the UI
    customization_window = widgets.VBox([plot_type_widget, udl_widget, matu_widget, param_widget, level_widget,
                                         window_widget, start_date_widget, end_date_widget,
                                         position_dropdown, plot_button, add_to_slide_button, output_plot])

    # Full app layout
    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

# Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [57]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = [(u, m, p, l) for u in udl for m in matu for p in param for l in level[p]]
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Rolling functions for calculations
def rolling_z_score(df, window=52):
    rolling_mean = df.rolling(window=window).mean()
    rolling_std = df.rolling(window=window).std()
    return (df - rolling_mean) / rolling_std

# Step 3: Create the custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(10, 6))
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plot 20th-80th percentile range as bars
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

    # Plot min/max as horizontal lines
    for i, category in enumerate(categories):
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

    # Plot average as triangles
    for i, category in enumerate(categories):
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

    # Plot last values as diamonds
    for i, category in enumerate(categories):
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.set_title("Custom Stats Chart")
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, frameon=False)
    plt.tight_layout()
    return fig

# Step 4: The main app with customizable plotting and placement
def create_app(df):
    # Widgets for user input
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=df.columns.get_level_values('UDL').unique().tolist(), description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0

    # Sidebar and Main Slide Display
    left_sidebar = widgets.VBox()
    main_slide_display = widgets.Output(layout=widgets.Layout(width="55%", height="600px", border="1px solid black"))

    # Function to preview the plot
    def on_plot_button_clicked(b):
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        # Filter DataFrame
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                return

        # Plot Preview based on selected plot type
        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = rolling_z_score(filtered_df, window=selected_window).fillna(0)
                fig, ax = plt.subplots(figsize=(6, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
            
            plt.show()
            # Attach figure to add button
            add_to_slide_button.fig = fig

        # Enable "Add to Slide" button
        add_to_slide_button.disabled = False

    # Function to add previewed plot to slide
    def on_add_to_slide_button_clicked(b):
        if not hasattr(b, 'fig'):
            return

        # Capture the current figure from the preview
        fig = b.fig
        position = position_dropdown.value
        plot_data = {"position": position, "figure": fig}

        # Add plot to selected slide
        slides[selected_slide_index]["plots"].append(plot_data)
        render_slide()
        update_sidebar()

    # Function to open selected slide
    def open_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    # Function to open selected plot in customization window
    def open_plot_for_edit(plot_data):
        # Load plot data back to customization view
        position_dropdown.value = plot_data["position"]
        output_plot.clear_output(wait=True)
        with output_plot:
            plot_data["figure"].canvas.draw()
            plt.show()

    # Attach button click events
    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)

    # Sidebar update
    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            checkbox = widgets.Checkbox(description=f"Slide {i+1}")
            checkbox.observe(lambda change, idx=i: open_slide(idx) if change['new'] else None, names='value')

            thumbnail = generate_slide_thumbnail(slide)
            img_widget = widgets.Image(value=thumbnail, format='png', width=100, height=100)
            slide_button = widgets.VBox([checkbox, img_widget])
            sidebar_content.append(slide_button)
        left_sidebar.children = sidebar_content

    # Function to generate slide thumbnail
    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        for plot_data in slide["plots"]:
            ax = axs[0][0] if plot_data["position"] == "top-left" else \
                 axs[0][1] if plot_data["position"] == "top-right" else \
                 axs[1][0] if plot_data["position"] == "bottom-left" else \
                 axs[1][1]
            ax.imshow(plot_data["figure"].canvas.buffer_rgba())
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()
        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        return buf.read()

    # Function to render the current slide
    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for plot_data in slide["plots"]:
                pos = (0, 0) if plot_data["position"] == "top-left" else \
                      (0, 1) if plot_data["position"] == "top-right" else \
                      (1, 0) if plot_data["position"] == "bottom-left" else \
                      (1, 1)
                ax = axs[pos]
                ax.imshow(plot_data["figure"].canvas.buffer_rgba())
            plt.tight_layout()
            plt.show()

    # Full app layout
    customization_window = widgets.VBox([plot_type_widget, udl_widget, matu_widget, param_widget, level_widget,
                                         window_widget, start_date_widget, end_date_widget,
                                         position_dropdown, plot_button, add_to_slide_button, output_plot])

    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

# Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [58]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = [(u, m, p, l) for u in udl for m in matu for p in param for l in level[p]]
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Rolling functions for calculations
def rolling_z_score(df, window=52):
    rolling_mean = df.rolling(window=window).mean()
    rolling_std = df.rolling(window=window).std()
    return (df - rolling_mean) / rolling_std

# Step 3: Create the custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(10, 6))
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plot 20th-80th percentile range as bars
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

    # Plot min/max as horizontal lines
    for i, category in enumerate(categories):
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

    # Plot average as triangles
    for i, category in enumerate(categories):
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

    # Plot last values as diamonds
    for i, category in enumerate(categories):
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.set_title("Custom Stats Chart")
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, frameon=False)
    plt.tight_layout()
    return fig

# Step 4: The main app with customizable plotting and placement
def create_app(df):
    # Widgets for user input
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=df.columns.get_level_values('UDL').unique().tolist(), description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    export_button = widgets.Button(description='Export to PPT', button_style='warning')
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0

    # Sidebar and Main Slide Display
    left_sidebar = widgets.VBox()
    main_slide_display = widgets.Output(layout=widgets.Layout(width="55%", height="600px", border="1px solid black"))

    # Function to preview the plot
    def on_plot_button_clicked(b):
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        # Filter DataFrame
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                return

        # Plot Preview based on selected plot type
        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = rolling_z_score(filtered_df, window=selected_window).fillna(0)
                fig, ax = plt.subplots(figsize=(6, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
            
            plt.show()
            # Attach figure to add button
            add_to_slide_button.fig = fig

        # Enable "Add to Slide" button
        add_to_slide_button.disabled = False

    # Function to add previewed plot to slide
    def on_add_to_slide_button_clicked(b):
        if not hasattr(b, 'fig'):
            return

        # Capture the current figure from the preview
        fig = b.fig
        position = position_dropdown.value
        plot_data = {"position": position, "figure": fig}

        # Add plot to selected slide
        slides[selected_slide_index]["plots"].append(plot_data)
        render_slide()
        update_sidebar()

    # Function to export all slides to PowerPoint
    def on_export_button_clicked(b):
        prs = Presentation()
        slide_layout = prs.slide_layouts[5]

        for slide in slides:
            slide_ppt = prs.slides.add_slide(slide_layout)
            for plot_data in slide["plots"]:
                buf = BytesIO()
                plot_data["figure"].savefig(buf, format='png')
                buf.seek(0)

                left = Inches(1) if plot_data["position"].startswith("top") else Inches(1)
                top = Inches(0.5) if plot_data["position"].endswith("left") else Inches(3)

                pic = slide_ppt.shapes.add_picture(buf, left, top, width=Inches(4), height=Inches(3))

        prs.save("exported_presentation.pptx")
        print("Slides exported successfully to 'exported_presentation.pptx'.")

    # Function to open selected slide
    def open_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    # Function to open selected plot in customization window
    def open_plot_for_edit(plot_data):
        # Load plot data back to customization view
        position_dropdown.value = plot_data["position"]
        output_plot.clear_output(wait=True)
        with output_plot:
            plot_data["figure"].canvas.draw()
            plt.show()

    # Attach button click events
    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)
    export_button.on_click(on_export_button_clicked)

    # Sidebar update
    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            checkbox = widgets.Checkbox(description=f"Slide {i+1}")
            checkbox.observe(lambda change, idx=i: open_slide(idx) if change['new'] else None, names='value')

            thumbnail = generate_slide_thumbnail(slide)
            img_widget = widgets.Image(value=thumbnail, format='png', width=100, height=100)
            slide_button = widgets.VBox([checkbox, img_widget])
            sidebar_content.append(slide_button)
        left_sidebar.children = sidebar_content

    # Function to generate slide thumbnail
    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        for plot_data in slide["plots"]:
            ax = axs[0][0] if plot_data["position"] == "top-left" else \
                 axs[0][1] if plot_data["position"] == "top-right" else \
                 axs[1][0] if plot_data["position"] == "bottom-left" else \
                 axs[1][1]
            ax.imshow(plot_data["figure"].canvas.buffer_rgba())
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()
        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        return buf.read()

    # Function to render the current slide
    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            for plot_data in slide["plots"]:
                pos = (0, 0) if plot_data["position"] == "top-left" else \
                      (0, 1) if plot_data["position"] == "top-right" else \
                      (1, 0) if plot_data["position"] == "bottom-left" else \
                      (1, 1)
                ax = axs[pos]
                ax.imshow(plot_data["figure"].canvas.buffer_rgba())
                ax.set_title(f"{plot_data['position']}")
                ax.set_xticks([])
                ax.set_yticks([])

                # Make plot clickable
                ax.figure.canvas.mpl_connect('button_press_event', lambda event, data=plot_data: open_plot_for_edit(data))
            plt.tight_layout()
            plt.show()

    # Full app layout
    customization_window = widgets.VBox([plot_type_widget, udl_widget, matu_widget, param_widget, level_widget,
                                         window_widget, start_date_widget, end_date_widget,
                                         position_dropdown, plot_button, add_to_slide_button, output_plot, export_button])

    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

# Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)


In [60]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches

# Step 1: Create the multi-index DataFrame generator
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = [(u, m, p, l) for u in udl for m in matu for p in param for l in level[p]]
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

# Step 2: Rolling functions for calculations
def rolling_z_score(df, window=52):
    rolling_mean = df.rolling(window=window).mean()
    rolling_std = df.rolling(window=window).std()
    return (df - rolling_mean) / rolling_std

# Step 3: Create the custom stats chart function
def create_stats_chart(df):
    fig, ax = plt.subplots(figsize=(10, 6))
    categories = df.columns.get_level_values('UDL').unique()
    min_values = df.min().values
    max_values = df.max().values
    avg_values = df.mean().values
    last_values = df.iloc[-1].values
    percentile_20 = df.quantile(0.2).values
    percentile_80 = df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    # Plot 20th-80th percentile range as bars
    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width, label='20th-80th %ile' if i == 0 else "")

    # Plot min/max as horizontal lines
    for i, category in enumerate(categories):
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3, label='Min/Max' if i == 0 else "")
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)

    # Plot average as triangles
    for i, category in enumerate(categories):
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Avg' if i == 0 else "")

    # Plot last values as diamonds
    for i, category in enumerate(categories):
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black', label='Last' if i == 0 else "")

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.set_title("Custom Stats Chart")
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4, frameon=False)
    plt.tight_layout()
    return fig

# Step 4: The main app with customizable plotting and placement
def create_app(df):
    # Widgets for user input
    plot_type_widget = widgets.Dropdown(options=["Time Series Plot", "Custom Stats Chart"], value="Custom Stats Chart", description="Plot Type:")
    udl_widget = widgets.SelectMultiple(options=df.columns.get_level_values('UDL').unique(), value=df.columns.get_level_values('UDL').unique().tolist(), description='UDL:')
    matu_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Matu').unique(), value=[df.columns.get_level_values('Matu').unique()[0]], description='Matu:')
    param_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Param').unique(), value=[df.columns.get_level_values('Param').unique()[0]], description='Param:')
    level_widget = widgets.SelectMultiple(options=df.columns.get_level_values('Level').unique(), value=[df.columns.get_level_values('Level').unique()[0]], description='Level:')
    window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:')
    start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime())
    end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime())
    plot_button = widgets.Button(description='Preview Plot', button_style='success')
    add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True)
    position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:")
    export_button = widgets.Button(description='Export to PPT', button_style='warning')
    output_plot = widgets.Output()
    slides = [{"plots": []}]
    selected_slide_index = 0

    # Sidebar and Main Slide Display
    left_sidebar = widgets.VBox()
    main_slide_display = widgets.Output(layout=widgets.Layout(width="55%", height="600px", border="1px solid black"))

    # Function to preview the plot
    def on_plot_button_clicked(b):
        plot_type = plot_type_widget.value
        selected_udl = list(udl_widget.value)
        selected_matu = list(matu_widget.value)
        selected_param = list(param_widget.value)
        selected_level = list(level_widget.value)
        selected_window = window_widget.value
        start_date = start_date_widget.value
        end_date = end_date_widget.value

        # Filter DataFrame
        filtered_df = df.loc[start_date:end_date, df.columns.get_level_values('UDL').isin(selected_udl) &
                                           df.columns.get_level_values('Matu').isin(selected_matu) &
                                           df.columns.get_level_values('Param').isin(selected_param) &
                                           df.columns.get_level_values('Level').isin(selected_level)]
        if filtered_df.empty:
            with output_plot:
                clear_output(wait=True)
                print("No valid data available for the selected combination. Please adjust your selection.")
                return

        # Plot Preview based on selected plot type
        with output_plot:
            clear_output(wait=True)
            if plot_type == "Time Series Plot":
                result_df = rolling_z_score(filtered_df, window=selected_window).fillna(0)
                fig, ax = plt.subplots(figsize=(6, 4))
                for column in result_df.columns:
                    ax.plot(result_df.index, result_df[column], label=str(column))
                ax.set_title("Time Series Plot Preview")
                ax.legend()
            elif plot_type == "Custom Stats Chart":
                fig = create_stats_chart(filtered_df)
            
            plt.show()
            # Attach figure to add button
            add_to_slide_button.fig = fig

        # Enable "Add to Slide" button
        add_to_slide_button.disabled = False

    # Function to add previewed plot to slide
    def on_add_to_slide_button_clicked(b):
        if not hasattr(b, 'fig'):
            return

        # Capture the current figure from the preview
        fig = b.fig
        position = position_dropdown.value
        plot_data = {"position": position, "figure": fig}

        # Add plot to selected slide
        slides[selected_slide_index]["plots"].append(plot_data)
        render_slide()
        update_sidebar()

    # Function to export all slides to PowerPoint
    def on_export_button_clicked(b):
        prs = Presentation()
        slide_layout = prs.slide_layouts[5]

        for slide in slides:
            slide_ppt = prs.slides.add_slide(slide_layout)
            for plot_data in slide["plots"]:
                buf = BytesIO()
                plot_data["figure"].savefig(buf, format='png')
                buf.seek(0)

                left = Inches(1) if plot_data["position"].startswith("top") else Inches(5)
                top = Inches(0.5) if plot_data["position"].endswith("left") else Inches(3.5)

                slide_ppt.shapes.add_picture(buf, left, top, width=Inches(4), height=Inches(3))

        prs.save("exported_presentation.pptx")
        print("Slides exported successfully to 'exported_presentation.pptx'.")

    # Function to open selected slide
    def open_slide(index):
        nonlocal selected_slide_index
        selected_slide_index = index
        render_slide()

    # Function to open selected plot in customization window
    def open_plot_for_edit(plot_data):
        # Load plot data back to customization view
        position_dropdown.value = plot_data["position"]
        output_plot.clear_output(wait=True)
        with output_plot:
            plot_data["figure"].canvas.draw()
            plt.show()

    # Attach button click events
    plot_button.on_click(on_plot_button_clicked)
    add_to_slide_button.on_click(on_add_to_slide_button_clicked)
    export_button.on_click(on_export_button_clicked)

    # Sidebar update
    def update_sidebar():
        sidebar_content = []
        for i, slide in enumerate(slides):
            checkbox = widgets.Checkbox(description=f"Slide {i+1}")
            checkbox.observe(lambda change, idx=i: open_slide(idx) if change['new'] else None, names='value')

            thumbnail = generate_slide_thumbnail(slide)
            img_widget = widgets.Image(value=thumbnail, format='png', width=100, height=100)
            slide_button = widgets.VBox([checkbox, img_widget])
            sidebar_content.append(slide_button)
        left_sidebar.children = sidebar_content

    # Function to generate slide thumbnail
    def generate_slide_thumbnail(slide):
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        for plot_data in slide["plots"]:
            ax = axs[0][0] if plot_data["position"] == "top-left" else \
                 axs[0][1] if plot_data["position"] == "top-right" else \
                 axs[1][0] if plot_data["position"] == "bottom-left" else \
                 axs[1][1]
            ax.imshow(plot_data["figure"].canvas.buffer_rgba())
            ax.set_xticks([])
            ax.set_yticks([])
        plt.tight_layout()
        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        return buf.read()

    # Function to render the current slide
    def render_slide():
        main_slide_display.clear_output(wait=True)
        with main_slide_display:
            slide = slides[selected_slide_index]
            fig, axs = plt.subplots(2, 2, figsize=(10, 8))
            position_map = {
                "top-left": (0, 0), "top-right": (0, 1),
                "bottom-left": (1, 0), "bottom-right": (1, 1)
            }
            for plot_data in slide["plots"]:
                pos = position_map[plot_data["position"]]
                ax = axs[pos]
                ax.imshow(plot_data["figure"].canvas.buffer_rgba())
                ax.set_title(f"{plot_data['position']}")
                ax.set_xticks([])
                ax.set_yticks([])

                # Set plot clickable
                def on_click(event, data=plot_data):
                    if ax.contains(event)[0]:  # Ensure correct subplot is clicked
                        open_plot_for_edit(data)

                fig.canvas.mpl_connect('button_press_event', on_click)

            plt.tight_layout()
            plt.show()

    # Full app layout
    customization_window = widgets.VBox([plot_type_widget, udl_widget, matu_widget, param_widget, level_widget,
                                         window_widget, start_date_widget, end_date_widget,
                                         position_dropdown, plot_button, add_to_slide_button, output_plot, export_button])

    display(widgets.HBox([left_sidebar, main_slide_display, customization_window]))
    slides.append({"plots": []})

# Example usage
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}

df = create_multi_index_dataframe(udl, matu, param, level)
create_app(df)
